In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

import sklearn
from sklearn import linear_model

%matplotlib inline

data = loadmat('data/ex3data1.mat')
data

In [ ]:
data['X'].shape, data['y'].shape

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def costReg(X, y, theta, lam):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (lam / 2 / len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

In [ ]:
def gradient(X, y, theta, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    gradient = np.zeros(X.shape[1])    
    error = sigmoid(X*theta.T) - y
    reg = learningRate / len(X) * np.sum(theta[:,1:theta.shape[1]], 2)
    
    gradient= 1/len(X) * X.T * error + reg
    gradient[0,0] = 1/len(X) * X.T * error
    return gradient

In [ ]:
plt.imshow(data['X'][0].reshape(20,20))

**One vs All Classifiers**

In [ ]:
X = data['X']
y = data['y']

In [ ]:
def one_vs_all(X, y, labels):
    X_one = np.insert(X, 0, values=np.ones(len(X)), axis=1)
    
    def classify(label):
        y_logical = (y == label)
        y_logical = y_logical.astype(int).reshape(len(y_logical))
        
        clf = linear_model.LogisticRegression(C=1.0)
        clf.fit(X_one,y_logical)
        return clf.coef_
    
    return np.array([classify(i) for i in range(1,labels+1)])
    

In [ ]:
def predictOneVsAll(X,y, theta):
    X_one = np.insert(X, 0, values=np.ones(len(X)), axis=1)
    X_one_matrix = np.matrix(X_one)
    
    # Plus 1 because labels are 1-based
    # arg max finds the high value ofo the column
    return np.argmax(X_one_matrix * np.matrix(theta).T, axis=1) + 1
    
    

In [ ]:
theta = one_vs_all(X,y,10)

In [ ]:
sklearn.metrics.accuracy_score(predictOneVsAll(X,y,theta), y)

### Neural Nets

In [ ]:
weights = loadmat('data/ex3weights.mat')
weights

In [ ]:
theta1 = weights['Theta1']
theta2 = weights['Theta2']

In [ ]:
X_one = np.insert(X, 0, values=np.ones(len(X)), axis=1)

In [ ]:
print(theta1.shape)
print(X_one.shape)

In [ ]:
(np.matrix(X_one) * np.matrix(theta1).T).shape

In [ ]:
def feedForward(X, theta):
    X_one = np.matrix(np.insert(X, 0, values=np.ones(len(X)), axis=1))
    theta_matrix = np.matrix(theta)
    
    return X_one * theta_matrix.T

In [ ]:
def feedForwardPredict(X, thetas):
    X_new = X.copy()
    for theta in thetas:
        X_new = feedForward(X_new, theta)
    
    return np.argmax(sigmoid(X_new), axis=1) + 1

In [ ]:
sklearn.metrics.accuracy_score(feedForwardPredict(X, [theta1, theta2]), y)